## Figure 5
Difference in CDNC between 3 surface-partitioning models and constant surface tension at +200m in parcel simulation. 
Columns show results from different models (OV, SL, RU).
Rows show slices through the 3-dimensional phase space of Na-w-Forg.

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install atmos-cloud-sim-uj-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
from PySDM_examples.Singer_Ward import Settings, ParcelSimulation
from PySDM_examples.Singer_Ward.aerosol import AerosolBetaCaryophylleneDark

from PySDM.initialisation.sampling import spectral_sampling as spec_sampling
from PySDM.physics import si

from open_atmos_jupyter_utils import show_plot

import numpy as np
import os
from matplotlib import pyplot
from matplotlib.colors import TwoSlopeNorm
from joblib import Parallel, delayed

import warnings
from numba.core.errors import NumbaExperimentalFeatureWarning

In [4]:
CI = 'CI' in os.environ
CI = True

updraft_list = np.geomspace(0.1, 10, 3 if CI else 8)
forg_list = np.linspace(0.05, 0.95, 3 if CI else 8)
models = ('Constant', 'CompressedFilmOvadnevaite', 'SzyszkowskiLangmuir', 'CompressedFilmRuehl')

In [5]:
def compute(key, settings):
    simulation = ParcelSimulation(settings)
    output = simulation.run()        
    output['updraft'] = settings.w
    output['org_fraction'] = settings.aerosol.modes[0]['f_org']
    output['color'] = settings.aerosol.color
    return key, output

print(f'tasks scheduled: {len(models) * len(forg_list) * len(updraft_list)}')
output = dict(Parallel(verbose=10, n_jobs=-1)(
    delayed(compute)(f"w{w:.2f}_f{Forg:.2f}_"+model, Settings(
        dz = 1 * si.m, 
        n_sd_per_mode = 100, 
        model = model,
        aerosol = AerosolBetaCaryophylleneDark(Forg=Forg, N=200),
        w = w * si.m / si.s,
        spectral_sampling = spec_sampling.ConstantMultiplicity
    ))
    for w in updraft_list
    for Forg in forg_list
    for model in models
))

tasks scheduled: 36


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/claresinger/Research/PySDM/PySDM/initialisation/equilibrate_wet_radii.py:84: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  fa = minfun(a, *args)
/Users/claresinger/Research/PySDM/PySDM/initialisation/equilibrate_wet_radii.py:84: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  fa = minfun(a, *args)
/Users/claresinger/Research/PySDM/PySDM/initialisation/equilibrate_wet_radii.py:84: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  fa = minfun(a, *args)
/Users/claresinger/Research/PySDM/PySDM/initialisation/equilibrate_wet_radii.py:84: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  fa = minfun(a, *args)
<string>:30: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
<string>:31: NumbaExperimentalFeatureWarning: First-class functi

In [ ]:
dCDNC = np.zeros((len(models), len(updraft_list), len(forg_list)))
for m in range(0,3):
    for i,w in enumerate(updraft_list):
        for j,Forg in enumerate(forg_list):
            key = f"w{w:.2f}_f{Forg:.2f}_"
            var = 'n_c_cm3'
            z = np.array(output[key+"Constant"]['z'])
            wz = np.where(z == z[-1])[0][0]
            CDNC_film = np.array(output[key+models[m+1]][var])[wz]
            CDNC_bulk = np.array(output[key+"Constant"][var])[wz]
            dCDNC[m,i,j] = (CDNC_film - CDNC_bulk) / CDNC_bulk * 100.0

pyplot.rcParams.update({"font.size":20})
fig, axes = pyplot.subplots(1, 4, figsize=(20,6), gridspec_kw={"width_ratios":[1,1,1,0.05]})

for i in range(0,3):
    ax = axes[i]
    ax.set_title(models[i+1])
    
    levs = np.linspace(-20,20,40)
    p = ax.contourf(forg_list, updraft_list, dCDNC[i], 
        cmap="bwr", levels=levs, extend="both")
    
    ax.set_yscale("log")
    if i == 0:
        ax.set_ylabel("Updraft [ms$^{-1}$]")
        ax.set_yticks([0.1,1,10])
        ax.set_yticklabels(["0.1","1","10"])
    elif i < 3:
        ax.set_yticklabels([])

fig.supxlabel("Organic fraction", y=-0.05)
pyplot.colorbar(p, cax=axes[-1], label=r"$\Delta_{CDNC}$ [%]")
pyplot.rcParams.update({'font.size': 15})
show_plot("fig5.pdf")